In [1]:
import h5py # .h5 파일을 읽기 위한 패키지
import random
import pandas as pd
import numpy as np
import os
import glob
import math
from time import time

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

from tqdm.auto import tqdm
from cfg.voxelnet_cfg import config as cfg 

from sklearn.metrics import accuracy_score

import warnings
warnings.filterwarnings(action='ignore') 

In [2]:
class VoxelDataset(Dataset) :
    def __init__(self, id_list, label_list, point_list) :
        self.id_list = id_list
        self.label_list = label_list
        self.point_list = point_list
    
    def __getitem__(self, index) :
        image_id = self.id_list[index]
        
        points= self.point_list[str(image_id)][:]
        voxel_features, voxel_coords= self.voxelization(points)

        if self.label_list is not None:
            label = self.label_list[index]
            return voxel_features, voxel_coords, label
        else:
            return voxel_features, voxel_coords
    
    def voxelization(self, point) :
        s = time()
        point_reflectance = np.zeros((point.shape[0],1))
        point = np.concatenate((point, point_reflectance), 1)
        
        voxel_coords = ((point[:, :3] - np.array([cfg.xrange[0], cfg.yrange[0], cfg.zrange[0]])) / 
                       (cfg.vw, cfg.vh, cfg.vd)).astype(np.int32)
        
        # convert to (D,H,W)
        voxel_coords = voxel_coords[:, [2,1,0]]
        voxel_coords, inv_ind, voxel_counts = np.unique(voxel_coords, 
                                                        axis=0, 
                                                        return_inverse=True, 
                                                        return_counts=True)  

        voxel_features = []
        for i in range(len(voxel_coords)) :
            voxel = np.zeros((cfg.T, 7), dtype=np.float32)
            pts = point[inv_ind == i]
            if voxel_counts[i] > cfg.T :
                pts = pts[:cfg.T, : ]
                voxel_counts[i] = cfg.T
            
            voxel[:pts.shape[0], :] = np.concatenate((pts, pts[:, :3] - np.mean(pts[:, :3], 0)), axis=1)
            voxel_features.append(voxel)

        return np.array(voxel_features), voxel_coords
    
    def __len__(self) :
        return len(self.id_list)

In [12]:
def detection_collate(batch):
    voxel_features = []
    voxel_coords = []
    train = True if len(batch[0]) == 3 else False
    
    for i, sample in enumerate(batch):
        voxel_features.append(sample[0])
        voxel_coords.append(np.pad(sample[1], ((0,0), (1,0)), mode='constant', constant_values=i))
            
    if train :
        print("Train")
        return np.concatenate(voxel_features), np.concatenate(voxel_coords), np.array(batch)[:, 2]
    else :
        print("Test")
        return np.concatenate(voxel_features), np.concatenate(voxel_coords), len(batch)
        

In [15]:
all_df = pd.read_csv('./data/train.csv')
all_points = h5py.File('./data/train.h5', 'r')

train_df = all_df.iloc[:int(len(all_df)*0.8)]
val_df = all_df.iloc[int(len(all_df)*0.8):]

train_dataset = VoxelDataset(train_df['ID'].values, train_df['label'].values, all_points)
# train_dataset = VoxelDataset(train_df['ID'].values, None, all_points)
train_loader = DataLoader(train_dataset, batch_size=64, collate_fn=detection_collate, shuffle=True, num_workers=0)


In [17]:
for vf,vc, lab in tqdm(iter(train_loader)) :
    print("vf.shape : ", vf.shape)
    print("vc.shape : ", vc.shape)
    print("lab.shape : ", np.array(lab).shape)
    break

  0%|          | 0/625 [00:00<?, ?it/s]

Train
vf.shape :  (14148, 35, 7)
vc.shape :  (14148, 4)
lab.shape :  (64,)
